This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [1]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *

from nltk.sem.logic import *
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

In [2]:
prover = Prover9()
#prover.config_prover9('/Users/champollion/Dropbox/oahzz')


In [3]:
from nltk.inference.prover9 import Prover9Exception
import traceback
# looks for a proof from just the premises provided
def try_to_prove_from(statement, premises_list, timeout_in_seconds=60):
    goal = statement
    prover = Prover9Command(goal, assumptions = premises_list, timeout = timeout_in_seconds)
    try:
        prover.prove()
        print(prover.proof())
    except Prover9Exception as e:
        traceback.print_exc()
        #print("Exceeded timeout of", timeout_in_seconds, "sec")

# looks for a proof based on all axioms in event_frame except for those in the exclusion list 
def try_to_prove_without(statement, exclusion_list, timeout_in_seconds=60):
    assert set(exclusion_list) <= set(event_frame)
    goal = statement
    prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not ((ax == statement) or ax in exclusion_list)], timeout = timeout_in_seconds)
    try:
        prover.prove()
        print(prover.proof())
    except Prover9Exception as e:
        traceback.print_exc()
        #print("Exceeded timeout of", timeout_in_seconds, "sec")

# looks for a proof based on all axioms in event_frame
def try_to_prove(statement, timeout_in_seconds=60):
    try_to_prove_without(statement,[], timeout_in_seconds)

# looks for a counterexample using Mace4 in which all the premises are true but the statement is false
def try_to_find_counterexample_from(statement, premise_list):
    goal = statement
    mb = MaceCommand(goal, assumptions = premise_list)
    print(mb.build_model())
    print(mb.model(format='cooked'))

# looks for a counterexample using Mace4 in which the statement is false and all the premises in
# event_frame are true with the possible exception of those in exclusion_list
# optionally specify domain size
def try_to_find_counterexample_without(statement, exclusion_list = [], size=-1):
    assert set(exclusion_list) <= set(event_frame)
    goal = statement
    assumptions_list = [ax for ax in event_frame if not ((ax == statement) or ax in exclusion_list)]
    if (size != -1):
        assumptions_list = [read_expr('assign(domain_size,'+str(size-1)+')')] + assumptions_list
    mb = MaceCommand(goal, assumptions = assumptions_list)
    print(mb.build_model())
    print(mb.model(format='cooked'))

# looks for a counterexample using Mace4 in which the statement is false and all the premises in
# event_frame-exclusion_list are true
def try_to_find_counterexample(statement):
    try_to_find_counterexample_without(statement, [])
    

# shows a model of the axioms in the list 
# optionally specify domain size
def show_example(premise_list, size=-1):
    if (size == -1):
        mb = MaceCommand(None, assumptions = premise_list)
    else:
        # for some reason we need to subtract 1 from the domain size
        mb = MaceCommand(None, [read_expr('assign(domain_size,'+str(size-1)+')')] + premise_list)
    print(mb.build_model())
    print(mb.model(format='cooked'))

## Axioms

### Lattice axioms

In [4]:
part_def = read_expr('Partof(x,y) <-> Product(x, y) = x')
proper_part_def = read_expr('Properpartof(x,y) <-> Partof(x,y) & -(x=y)')

compl_def = read_expr('Complementof(x,y) <-> (Product(x, y) = Bot & Sum(x, y) = Top)')


# the following definitions are formulated as implications so that they don't entail the existence of Bot and Top
# but only name them
bot_def = read_expr('(exists y. all x. Partof(y,x)) -> Partof(Bot,x)')
top_def = read_expr('(exists y. all x. Partof(x,y)) -> Partof(x, Top)')

#By including a definition of the covering relation,
#one can quickly read off Hasse diagrams of lattices from the Mace4 output.
# (Tip from Wes Holliday)
cover_def = read_expr('covered_by(x,y) <-> (-(x=y) & x = Sum(x,y) & -exists z.(-(z=x) & -(z=y) & x = Sum(x,z) & z = Sum(z,y)))')



sum_id = read_expr('Sum(x,x) = x')
product_id = read_expr('Product(x,x) = x')

sum_comm = read_expr('Sum(x,y) = Sum(y,x)')
product_comm = read_expr('Product(x,y) = Product(y,x)')

sum_assoc = read_expr('Sum(x,Sum(y,z)) = Sum(Sum(x,y),z)')
product_assoc = read_expr('Product(x,Product(y,z)) = Product(Product(x,y),z)')

sum_absorp = read_expr('Sum(x,Product(x,y)) = x')
product_absorp = read_expr('Product(x,Sum(x,y)) = x')

lattice = [cover_def, part_def, proper_part_def, bot_def, top_def, compl_def, sum_id, product_id, sum_comm, product_comm, sum_assoc, product_assoc, sum_absorp, product_absorp]

In [5]:
show_example(lattice,4)

True
% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

domain_size = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(0,2) = 0.
Product(0,3) = 0.
Product(1,0) = 0.
Product(1,1) = 1.
Product(1,2) = 2.
Product(1,3) = 3.
Product(2,0) = 0.
Product(2,1) = 2.
Product(2,2) = 2.
Product(2,3) = 0.
Product(3,0) = 0.
Product(3,1) = 3.
Product(3,2) = 0.
Product(3,3) = 3.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(0,2) = 2.
Sum(0,3) = 3.
Sum(1,0) = 1.
Sum(1,1) = 1.
Sum(1,2) = 1.
Sum(1,3) = 1.
Sum(2,0) = 2.
Sum(2,1) = 1.
Sum(2,2) = 2.
Sum(2,3) = 1.
Sum(3,0) = 3.
Sum(3,1) = 1.
Sum(3,2) = 1.
Sum(3,3) = 3.

f1(0,0) = 0.
f1(0,1) = 0.
f1(0,2) = 0.
f1(0,3) = 0.
f1(1,0) = 2.
f1(1,1) = 0.
f1(1,2) = 0.
f1(1,3) = 0.
f1(2,0) = 0.
f1(2,1) = 0.
f1(2,2) = 0.
f1(2,3) = 0.
f1(3,0) = 0.
f1(3,1) = 0.
f1(3,2) = 0.
f1(3,3) = 0.

f2(0,0) = 0.
f2(0,1) = 0.
f2(0,2) = 0.
f2(0,3) = 0.
f2(1,0) = 0.
f2(1,1) = 0.
f2(1,2) = 0.
f2(1,3) = 0.
f2(2,0) = 0.
f2(2,1) = 0.
f2(2,2) = 0.
f2(2,3) = 0.
f2(3,0) = 0.
f2(3,1) =

In [6]:
# idempotence of sum and product follow from absorption
try_to_prove_from(product_id, [sum_absorp,  product_absorp])

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 15860 was started by champollion on standingdesk.home,
Sun Jul 17 15:57:35 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 6.
% Level of proof is 2.
% Maximum clause weight is 7.000.
% Given clauses 2.


1 Product(x,x) = x.  [goal].
2 Sum(x,Product(x,y)) = x.  [assumption].
3 Product(x,Sum(x,y)) = x.  [assumption].
4 Product(c1,c1) != c1.  [deny(1)].
6 Product(x,x) = x.  [para(2(a,1),3(a,1,2))].
7 $F.  [resolve(6,a,4,a)].

============================== end of proof ==========================


In [7]:
try_to_prove_from(sum_id, [sum_absorp,product_absorp])

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 15862 was started by champollion on standingdesk.home,
Sun Jul 17 15:57:35 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 6.
% Level of proof is 2.
% Maximum clause weight is 7.000.
% Given clauses 2.


1 Sum(x,x) = x.  [goal].
2 Sum(x,Product(x,y)) = x.  [assumption].
3 Product(x,Sum(x,y)) = x.  [assumption].
4 Sum(c1,c1) != c1.  [deny(1)].
5 Sum(x,x) = x.  [para(3(a,1),2(a,1,2))].
6 $F.  [resolve(5,a,4,a)].

============================== end of proof ==========================


In [8]:
# Proof that parthood is transitive
transitivity_of_parthood = read_expr('Partof(x,y) & Partof(y,z) -> Partof(x,z)')
try_to_prove_from(transitivity_of_parthood, lattice)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 15864 was started by champollion on standingdesk.home,
Sun Jul 17 15:57:35 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 22.
% Level of proof is 8.
% Maximum clause weight is 11.000.
% Given clauses 40.


2 Partof(x,y) <-> Product(x,y) = x.  [assumption].
7 Partof(x,y) & Partof(y,z) -> Partof(x,z).  [goal].
21 -Partof(x,y) | Product(x,y) = x.  [clausify(2)].
22 Partof(x,y) | Product(x,y) != x.  [clausify(2)].
27 Sum(x,y) = Sum(y,x).  [assumption].
28 Product(x,y) = Product(y,x).  [assumption].
31 Product(x,Product(y,z)) = Product(Product(x,y),z).  [assumption].

In [9]:
def upper_bound(x,P):
    return '(all x1 ('+P+'(x1) -> Partof(x1,'+x+')))'

def least_upper_bound(x,P):
    return upper_bound(x,P) + ' & all y1 (('+upper_bound('y1','P')+') -> Partof('+x+',y1))'

def lower_bound(x,P):
    return '(all x2 ('+P+'(x2) -> Partof('+x+',x2)))'

def greatest_lower_bound(x,P):
    return lower_bound(x,P) + ' & all y2 (('+lower_bound('y2','P')+') -> Partof(y2,'+x+'))'

In [10]:
# Proof that sums are unique wherever they exist
unique_sum = read_expr('(('+least_upper_bound('x','P')+' & '+least_upper_bound('y','P')+') -> x=y)')
print(unique_sum)
try_to_prove_from(unique_sum,lattice)

((all x1.(P(x1) -> Partof(x1,x)) & all y1.(all x1.(P(x1) -> Partof(x1,y1)) -> Partof(x,y1)) & all x1.(P(x1) -> Partof(x1,y)) & all y1.(all x1.(P(x1) -> Partof(x1,y1)) -> Partof(y,y1))) -> (x = y))
============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 15866 was started by champollion on standingdesk.home,
Sun Jul 17 15:57:35 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 5.
% Maximum clause weight is 8.000.
% Given clauses 26.


2 Partof(x,y) <-> Product(x,y) = x.  [assumption].
7 (all x (P(x) -> Partof(x,y))) & (all z ((all x (P(x) -> Partof(x,z))) -> Partof(y,z))) & (all x (P(x) -> Partof(x

In [11]:
# Proof that products are unique wherever they exist
unique_product = read_expr('(('+greatest_lower_bound('x','P')+' & '+greatest_lower_bound('y','P')+') -> x=y)')
print(unique_product)
try_to_prove_from(unique_product,lattice)

((all x2.(P(x2) -> Partof(x,x2)) & all y2.(all x2.(P(x2) -> Partof(y2,x2)) -> Partof(y2,x)) & all x2.(P(x2) -> Partof(y,x2)) & all y2.(all x2.(P(x2) -> Partof(y2,x2)) -> Partof(y2,y))) -> (x = y))
============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 15868 was started by champollion on standingdesk.home,
Sun Jul 17 15:57:35 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 5.
% Maximum clause weight is 8.000.
% Given clauses 26.


2 Partof(x,y) <-> Product(x,y) = x.  [assumption].
7 (all x (P(x) -> Partof(y,x))) & (all z ((all x (P(x) -> Partof(z,x))) -> Partof(z,y))) & (all x (P(x) -> Partof(u

### Bounded lattice axioms

In [12]:
bounded = read_expr('(exists y. all x. Partof(y,x)) & (exists y. all x. Partof(x,y))')

has_bot = read_expr('(exists y. all x. Partof(y,x))')
has_top = read_expr('(exists y. all x. Partof(x,y))')

bounded_lattice = [bounded] + lattice

### Distributive lattice axioms

In [13]:
dist = read_expr('Product(x,Sum(y,z)) = Sum(Product(x,y), Product(x,z))')

dist_lattice = [dist] + lattice
bounded_dist_lattice = [bounded] + [dist] + lattice

In [14]:
#The other distributive lattice statement holds as a theorem
dist2 = read_expr('Sum(x,Product(y,z)) = Product(Sum(x,y), Sum(x,z))')
try_to_prove_from(dist2,dist_lattice)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 15870 was started by champollion on standingdesk.home,
Sun Jul 17 15:57:35 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 11.
% Level of proof is 3.
% Maximum clause weight is 15.000.
% Given clauses 12.


7 Sum(x,Product(y,z)) = Product(Sum(x,y),Sum(x,z)).  [goal].
21 Product(x,Sum(y,z)) = Sum(Product(x,y),Product(x,z)).  [assumption].
27 Product(x,x) = x.  [assumption].
29 Product(x,y) = Product(y,x).  [assumption].
30 Sum(x,Sum(y,z)) = Sum(Sum(x,y),z).  [assumption].
31 Sum(Sum(x,y),z) = Sum(x,Sum(y,z)).  [copy(30),flip(a)].
34 Sum(x,Product(x,y)) = x.  [assu

### Complete lattice axioms

In [15]:
## We would ideally like to define complete lattices, but the following doesn't work.
## This is because we can't properly universally quantify over predicates. 

exist_sum = read_expr('all P. (exists z.'+least_upper_bound('z','P')+')')
exist_product = read_expr('all Q.(exists z.'+greatest_lower_bound('z','Q')+')')

# this only guarantees existence and not uniqueness, 
# but as we have seen above, sums and products are unique wherever they exist

# A complete lattice is a lattice where every set has both 
# a sum (least upper bound) and a product (greatest lower bound).
complete = [exist_sum, exist_product]

## A bounded lattice is a lattice that has both a top (greatest element) and a bottom (least element).
## Every complete lattice is bounded. 
## So normally we should be able to prove boundedness from completeness, but
## this doesn't work. So we define complete lattices as complete bounded lattices.
complete_lattice = complete + bounded_lattice
complete_dist_lattice = complete + [dist] + bounded_lattice

In [16]:
#try_to_prove_from(bounded,complete_lattice) # times out
#try_to_find_counterexample_from(bounded, complete_lattice) # times out

### Boolean algebra axioms

In [17]:
# Any non-empty finite lattice is trivially complete.
# Still, the system finds a counterexample. 
# Here, the lattice that has a bottom (0), a top (1), and a single element (2) sitting between them
# is bounded and distributive. It is also complete but it is found as a counterexample to completeness.
# The predicate P is given as not applying to anything.
print(exist_sum)
try_to_find_counterexample_from(exist_sum,bounded_lattice)

all P.(exists z.all x1.(P(x1) -> Partof(x1,z)) & all y1.(all x1.(P(x1) -> Partof(x1,y1)) -> Partof(z,y1)))
True
% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

c2 = 1.

c3 = 1.

c5 = 0.

f4(0) = 0.
f4(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f1(0,0) = 0.
f1(0,1) = 0.
f1(1,0) = 0.
f1(1,1) = 0.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f3(0,0) = 0.
f3(0,1) = 0.
f3(1,0) = 0.
f3(1,1) = 0.

- P(0).
- P(1).

- Complementof(0,0).
  Complementof(0,1).
  Complementof(1,0).
- Complementof(1,1).

  Partof(0,0).
  Partof(0,1).
- Partof(1,0).
  Partof(1,1).

- Properpartof(0,0).
  Properpartof(0,1).
- Properpartof(1,0).
- Properpartof(1,1).

- covered_by(0,0).
- covered_by(0,1).
  covered_by(1,0).
- covered_by(1,1).



In [18]:
#complemented = read_expr('all x.(exists y. (Product(x, y) = Bot & Sum(x, y) = Top))')
complemented = read_expr('all x.(exists y. Complementof(x,y))')


#sectional complement
# We call the lattice L sectionally complemented if it has a zero and 
# for all x ≤ y ∈ L, there exists an element z ∈ L satisfying x ∨ z = y and x ∧ z = 0.
sec_compl = read_expr('all x. (all y. (Partof(x,y) -> exists z. (Sum(x, z) = y & Product(x, z) = Bot)))')

# A complemented lattice is a bounded lattice in which every element has a complement. 
complemented_lattice = [complemented] + bounded_lattice

# A boolean lattice is a complemented distributive lattice. 
boolean_lattice = [dist] + complemented_lattice  
boolean_algebra = boolean_lattice

complete_boolean_lattice = complete + boolean_lattice
complete_boolean_algebra = complete_boolean_lattice

In [19]:
show_example(boolean_lattice,1)

True
% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

domain_size = 0.

c1 = 0.

c2 = 1.

f1(0) = 1.
f1(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f2(0,0) = 0.
f2(0,1) = 0.
f2(1,0) = 0.
f2(1,1) = 0.

f3(0,0) = 0.
f3(0,1) = 0.
f3(1,0) = 0.
f3(1,1) = 0.

f4(0,0) = 0.
f4(0,1) = 0.
f4(1,0) = 0.
f4(1,1) = 0.

- Complementof(0,0).
  Complementof(0,1).
  Complementof(1,0).
- Complementof(1,1).

  Partof(0,0).
  Partof(0,1).
- Partof(1,0).
  Partof(1,1).

- Properpartof(0,0).
  Properpartof(0,1).
- Properpartof(1,0).
- Properpartof(1,1).

  assign(0,0).
- assign(0,1).
- assign(1,0).
- assign(1,1).

- covered_by(0,0).
- covered_by(0,1).
  covered_by(1,0).
- covered_by(1,1).



In [20]:
# Is every boolean lattice is sectionally complemented?
# try_to_prove_from(sec_compl, boolean_lattice) # times out

In [21]:
# Not every bounded lattice is complemented.
try_to_find_counterexample_from(complemented, bounded_lattice)
# size 3 counterexample. Bot=0, Top=1, 2 is in between. 
# this is bounded because every finite lattice is bounded.
# It is not complemented though: 2 has no complement.

True
% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 0.

c2 = 1.

c3 = 2.

Product(0,0) = 0.
Product(0,1) = 0.
Product(0,2) = 0.
Product(1,0) = 0.
Product(1,1) = 1.
Product(1,2) = 2.
Product(2,0) = 0.
Product(2,1) = 2.
Product(2,2) = 2.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(0,2) = 2.
Sum(1,0) = 1.
Sum(1,1) = 1.
Sum(1,2) = 1.
Sum(2,0) = 2.
Sum(2,1) = 1.
Sum(2,2) = 2.

f1(0,0) = 0.
f1(0,1) = 0.
f1(0,2) = 0.
f1(1,0) = 2.
f1(1,1) = 0.
f1(1,2) = 0.
f1(2,0) = 0.
f1(2,1) = 0.
f1(2,2) = 0.

f2(0,0) = 0.
f2(0,1) = 0.
f2(0,2) = 0.
f2(1,0) = 0.
f2(1,1) = 0.
f2(1,2) = 0.
f2(2,0) = 0.
f2(2,1) = 0.
f2(2,2) = 0.

f3(0,0) = 0.
f3(0,1) = 0.
f3(0,2) = 0.
f3(1,0) = 0.
f3(1,1) = 0.
f3(1,2) = 0.
f3(2,0) = 0.
f3(2,1) = 0.
f3(2,2) = 0.

- Complementof(0,0).
  Complementof(0,1).
- Complementof(0,2).
  Complementof(1,0).
- Complementof(1,1).
- Complementof(1,2).
- Complementof(2,0).
- Complementof(2,1).
- Complementof(2,2).

  Partof(0,0).
  Partof(0,1).
  Partof(0,2).
- Partof(1,0)

In [22]:
# Also not every bounded lattice is sectionally complemented.
try_to_find_counterexample_from(sec_compl, bounded_lattice)
# size 3 counterexample. Bot=0, Top=1, 2 is in between. 
# this is bounded because every finite lattice is bounded.
# It is not complemented though: 2 has no complement at all, so it has no sectional complement.

True
% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 0.

c2 = 1.

c3 = 2.

c4 = 1.

Product(0,0) = 0.
Product(0,1) = 0.
Product(0,2) = 0.
Product(1,0) = 0.
Product(1,1) = 1.
Product(1,2) = 2.
Product(2,0) = 0.
Product(2,1) = 2.
Product(2,2) = 2.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(0,2) = 2.
Sum(1,0) = 1.
Sum(1,1) = 1.
Sum(1,2) = 1.
Sum(2,0) = 2.
Sum(2,1) = 1.
Sum(2,2) = 2.

f1(0,0) = 0.
f1(0,1) = 0.
f1(0,2) = 0.
f1(1,0) = 2.
f1(1,1) = 0.
f1(1,2) = 0.
f1(2,0) = 0.
f1(2,1) = 0.
f1(2,2) = 0.

f2(0,0) = 0.
f2(0,1) = 0.
f2(0,2) = 0.
f2(1,0) = 0.
f2(1,1) = 0.
f2(1,2) = 0.
f2(2,0) = 0.
f2(2,1) = 0.
f2(2,2) = 0.

f3(0,0) = 0.
f3(0,1) = 0.
f3(0,2) = 0.
f3(1,0) = 0.
f3(1,1) = 0.
f3(1,2) = 0.
f3(2,0) = 0.
f3(2,1) = 0.
f3(2,2) = 0.

- Complementof(0,0).
  Complementof(0,1).
- Complementof(0,2).
  Complementof(1,0).
- Complementof(1,1).
- Complementof(1,2).
- Complementof(2,0).
- Complementof(2,1).
- Complementof(2,2).

  Partof(0,0).
  Partof(0,1).
  Partof(0,2).
- Pa

### Exclusion axioms

In [23]:
# From here on in we go beyond standard lattice theory and introduce an exclusion relation
# that is similar but not identical to the one in orthologic.
cumulativity = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(Sum(x1, x2), Sum(y1, y2))')
symmetry = read_expr('Excl(x, y) <-> Excl(y, x)')

### Possibility axioms

In [63]:
#define conflict:
confl_def = read_expr('Confl(x, y) <-> exists z1.(Partof(z1,x) & exists z2.(Partof(z2,y) & Excl(z1, z2)))')

# define possible:
poss_def = read_expr('Poss(x) <-> -Confl(x, x)')

#define possible world:
world_def = read_expr('World(x) <-> (Poss(x) & all y.(Properpartof(x,y) -> -Poss(y)))')

#Harmony
harmony = read_expr('all x.(all y. ((World(x) & -Confl(x, y)) -> Poss(y)))')

#Nirvana
nirvana = read_expr('all x.(-Confl(Bot, x))')

#Rashōmon
rashomon = read_expr('all x. (all y.((Poss(x) & Poss(y) & -Confl(x, y)) -> Poss(Sum(x,y))))')


#Cosmopolitanism
#cosmopol = read_expr('all x.(int_poss(x) -> exists y.(world(y) & Product(x, y) = y))')

cosmopol = read_expr('all x.(Poss(x) -> exists y.(World(y) & Partof(x,y)))')


#actual world
actual_world = read_expr('World(actual_world)')

#define actual:
actual_def = read_expr('(exists y. y = actual_world) <-> (all x. (Actual(x) <-> Partof(x, actual_world)))')

exclusion_axioms = [cumulativity, symmetry, harmony, nirvana, rashomon, cosmopol, actual_world]

#Construct event frame
#choice point: definition of possibility
#choice point: formulation of harmony

definitions = [confl_def, poss_def, world_def, actual_def]

nonnullworld = read_expr('-(World(Bot))') # Bottom is not a possible world. (If it was, it would be the only one.)
event_frame = complete_boolean_algebra + exclusion_axioms + definitions 
#event_frame = event_frame + [nonnullworld] # comment out if desired


nonnullexcl = read_expr('(exists x. exists y. Excl(x,y))')
#event_frame = event_frame + [nonnullexcl] # comment out if desired

canonical = read_expr('(all x. all y. all z. (((Excl(x,y) & Excl(x,z)) -> (y = z))))')
noncanonical = read_expr('-(all x. all y. all z. (((Excl(x,y) & Excl(x,z)) -> (y = z))))')
#event_frame = event_frame + [noncanonical] # comment out if desired


## Looking at some sample models

In [66]:
show_example(event_frame)
# size 2 interpretation, Bot=0, Top=1, actual_world=1, empty exclusion relation

True
% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

actual_world = 1.

c1 = 0.

c2 = 1.

c3 = 1.

c4 = 0.

f5(0) = 1.
f5(1) = 0.

f9(0) = 1.
f9(1) = 1.

f12(0) = 1.
f12(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f1(0,0) = 1.
f1(0,1) = 1.
f1(1,0) = 1.
f1(1,1) = 1.

f3(0,0) = 0.
f3(0,1) = 0.
f3(1,0) = 0.
f3(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 0.
f6(1,1) = 0.

f7(0,0) = 0.
f7(0,1) = 0.
f7(1,0) = 0.
f7(1,1) = 0.

f8(0,0) = 0.
f8(0,1) = 0.
f8(1,0) = 0.
f8(1,1) = 0.

f10(0,0) = 0.
f10(0,1) = 0.
f10(1,0) = 0.
f10(1,1) = 0.

f11(0,0) = 0.
f11(0,1) = 0.
f11(1,0) = 0.
f11(1,1) = 0.

f2(0,0,0) = 0.
f2(0,0,1) = 0.
f2(0,1,0) = 0.
f2(0,1,1) = 0.
f2(1,0,0) = 1.
f2(1,0,1) = 0.
f2(1,1,0) = 1.
f2(1,1,1) = 0.

f4(0,0,0) = 0.
f4(0,0,1) = 0.
f4(0,1,0) = 0.
f4(0,1,1) = 0.
f4(1,0,0) = 0.
f4(1,0,1) = 0.
f4(1,1,0) = 0.
f4(1,1,1) = 0.

  Actual(0).
  Actual(1).

  P(0).
  P(1).

In [69]:
show_example(event_frame,3)
# size 4 interpretation, Bot=0, Top=1, actual_world=1, everything is actual and possible, nothing 
# except 1 (Top) is a world, empty exclusion relation

True
% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

actual_world = 1.

domain_size = 0.

c1 = 0.

c2 = 1.

c3 = 1.

c4 = 0.

f5(0) = 1.
f5(1) = 0.
f5(2) = 3.
f5(3) = 2.

f9(0) = 1.
f9(1) = 1.
f9(2) = 1.
f9(3) = 1.

f12(0) = 1.
f12(1) = 0.
f12(2) = 1.
f12(3) = 1.

Product(0,0) = 0.
Product(0,1) = 0.
Product(0,2) = 0.
Product(0,3) = 0.
Product(1,0) = 0.
Product(1,1) = 1.
Product(1,2) = 2.
Product(1,3) = 3.
Product(2,0) = 0.
Product(2,1) = 2.
Product(2,2) = 2.
Product(2,3) = 0.
Product(3,0) = 0.
Product(3,1) = 3.
Product(3,2) = 0.
Product(3,3) = 3.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(0,2) = 2.
Sum(0,3) = 3.
Sum(1,0) = 1.
Sum(1,1) = 1.
Sum(1,2) = 1.
Sum(1,3) = 1.
Sum(2,0) = 2.
Sum(2,1) = 1.
Sum(2,2) = 2.
Sum(2,3) = 1.
Sum(3,0) = 3.
Sum(3,1) = 1.
Sum(3,2) = 1.
Sum(3,3) = 3.

f1(0,0) = 1.
f1(0,1) = 1.
f1(0,2) = 1.
f1(0,3) = 1.
f1(1,0) = 1.
f1(1,1) = 1.
f1(1,2) = 1.
f1(1,3) = 1.
f1(2,0) = 1.
f1(2,1) = 1.
f1(2,2) = 1.
f1(2,3) = 1.
f1(3,0) = 1.
f1(3,1) = 1.
f1(3,2) = 1

In [72]:
show_example(event_frame+[nonnullexcl],3) # example with nonnull exclusion relation
# size 4, Bot=0, Top=1, actual_world=2, everything except Bot excludes and is excluded by everything,
# only 2 is a world, only Bot and 2 are possible

True
% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

actual_world = 2.

domain_size = 0.

c1 = 0.

c2 = 1.

c3 = 2.

c4 = 0.

c5 = 1.

c6 = 1.

f5(0) = 1.
f5(1) = 0.
f5(2) = 3.
f5(3) = 2.

f9(0) = 2.
f9(1) = 0.
f9(2) = 2.
f9(3) = 0.

f12(0) = 2.
f12(1) = 0.
f12(2) = 0.
f12(3) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(0,2) = 0.
Product(0,3) = 0.
Product(1,0) = 0.
Product(1,1) = 1.
Product(1,2) = 2.
Product(1,3) = 3.
Product(2,0) = 0.
Product(2,1) = 2.
Product(2,2) = 2.
Product(2,3) = 0.
Product(3,0) = 0.
Product(3,1) = 3.
Product(3,2) = 0.
Product(3,3) = 3.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(0,2) = 2.
Sum(0,3) = 3.
Sum(1,0) = 1.
Sum(1,1) = 1.
Sum(1,2) = 1.
Sum(1,3) = 1.
Sum(2,0) = 2.
Sum(2,1) = 1.
Sum(2,2) = 2.
Sum(2,3) = 1.
Sum(3,0) = 3.
Sum(3,1) = 1.
Sum(3,2) = 1.
Sum(3,3) = 3.

f1(0,0) = 1.
f1(0,1) = 1.
f1(0,2) = 1.
f1(0,3) = 1.
f1(1,0) = 1.
f1(1,1) = 1.
f1(1,2) = 1.
f1(1,3) = 1.
f1(2,0) = 1.
f1(2,1) = 1.
f1(2,2) = 1.
f1(2,3) = 1.
f1(3,0) = 1.
f1(3,1

## Independence of axioms

In [30]:
# assumptions tested for exclusion: actual_world, cosmopol, complemented, cumulativity, dist, harmony, rashomon
# If we don't assume Nirvana, Mace4 finds a counterexample that shows symmetry is an axiom.
try_to_find_counterexample_without(symmetry,[nirvana])
# In that counterexample, the null event is a possible world. If we rule this out, 
# the system finds no counterexamples.

True
% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

actual_world = 0.

c1 = 0.

c2 = 1.

c3 = 0.

c4 = 0.

c5 = 0.

c6 = 1.

f5(0) = 1.
f5(1) = 0.

f9(0) = 0.
f9(1) = 0.

f12(0) = 0.
f12(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f1(0,0) = 1.
f1(0,1) = 1.
f1(1,0) = 1.
f1(1,1) = 1.

f3(0,0) = 0.
f3(0,1) = 0.
f3(1,0) = 0.
f3(1,1) = 0.

f6(0,0) = 0.
f6(0,1) = 0.
f6(1,0) = 0.
f6(1,1) = 0.

f7(0,0) = 0.
f7(0,1) = 0.
f7(1,0) = 0.
f7(1,1) = 0.

f8(0,0) = 0.
f8(0,1) = 0.
f8(1,0) = 0.
f8(1,1) = 0.

f10(0,0) = 0.
f10(0,1) = 0.
f10(1,0) = 0.
f10(1,1) = 0.

f11(0,0) = 0.
f11(0,1) = 1.
f11(1,0) = 0.
f11(1,1) = 1.

f2(0,0,0) = 0.
f2(0,0,1) = 0.
f2(0,1,0) = 0.
f2(0,1,1) = 0.
f2(1,0,0) = 1.
f2(1,0,1) = 0.
f2(1,1,0) = 1.
f2(1,1,1) = 0.

f4(0,0,0) = 0.
f4(0,0,1) = 0.
f4(0,1,0) = 0.
f4(0,1,1) = 0.
f4(1,0,0) = 0.
f4(1,0,1) = 0.
f4(1,1,0) = 0.
f4(1,1,1) = 0.

  Actual(0).
- Actual(1)

In [ ]:
# If we do assume Nirvana, it doesn't find that counterexample. So symmetry might be a theorem.
try_to_find_counterexample_without(symmetry,[])  # times out
# This doesn't depend on whether we disallow the null event from being a possible world.

In [ ]:
# Even though symmetry might be a theorem, we can't know for sure because the search for a proof also times out.
#try_to_prove_without(symmetry,[]) # times out
# Again this doesn't depend on whether nonnullworld is included.

In [ ]:
# assumptions tested for exclusion: 
# actual_world, cosmopol, complemented, cumulativity, dist, harmony, nirvana, rashomon

# If we don't assume Nirvana, Mace4 finds a counterexample that shows cumulativity is an axiom.
# In that counterexample there are just two elements: Bot and Top. Bot is the actual world, 
# Bot and Top exclude each other but not themselves.
# Insisting on nonnullworld rules out this counterexample and causes the search to time out.
try_to_find_counterexample_without(cumulativity,[nirvana])
# assumptions tested for exclusion: actual_world, cosmopol, complemented, dist, harmony, rashomon

In [ ]:
#There is also a counterexample if we include Nirvana but not Complemented. 
# Size 3. Bot=0, Top=1, actual_world = 2. No worlds other than actual_world. 
# Top is impossible. Actual_world and Top exclude each other. Top conflicts with itself but doesn't exclude itself.
try_to_find_counterexample_without(cumulativity,[complemented])

In [ ]:
# If we insist on Nirvana and Complemented, Mace4 doesn't find a counterexample:
#try_to_find_counterexample_without(cumulativity,[]) # times out

In [ ]:
# But if we try to prove cumulativity, it also times out:
#try_to_prove(cumulativity) # times out
# So we don't know if cumulativity is a theorem. If it isn't, the counterexamples are likely infinitely sized.

In [ ]:
# If we try to prove symmetry it times out:
#try_to_prove(symmetry) # times out

In [ ]:
#try_to_find_counterexample_without(symmetry,[]) # times out so we don't know if symmetry is an axiom

In [ ]:
try_to_find_counterexample_without(symmetry,[harmony]) 
# succeeds -- so if symmetry is a theorem then it depends on harmony.
# In this counterexample, both Harmony and Symmetry fail.
# Two elements: only Bot and Top. Top excludes Bot but not the other way around (so Symmetry fails).
# So Top conflicts with Bot but not the other way around. Since Bot is a part of Top, Top conflicts with itself.
# So Bot is possible, and in fact is a possible world. Top is not possible.
# Harmony says that if a possible world conforms with (i.e. fails to conflict with) an event, that event is possible.
# Here, Bot is a possible world, and Bot does not conflict with Top (even though Top conflicts with Bot). 
# Yet Top is impossible because it conflicts with itself.
# (If nonnullworld is switched on, this search times out. )

In [ ]:
try_to_find_counterexample(harmony) # succeeds as expected
# Same counterexample as above except that this time, Top excludes Top instead of excluding Bot. 
# No further exclusions hold. As a result, Top conflicts with itself and there are no other conflicts.
# So Bot is a possible world. It doesn't conflict with Top even though Top is impossible.
# By Harmony, since Bot doesn't conflict with Top, Top should be possible.

# Even if nonnullworld is switched on, this succeeds. In that case there are four elements.
# Bot=0, actual_world=1, Top=2, only 0 and 1 are possible, exclusions: 2-2, 2-3, 3-3.

In [ ]:
#try_to_prove(harmony) # times out -- as expected

In [ ]:
#try_to_find_counterexample(cosmopol) # fails as expected because countermodels are infinite

In [ ]:
#try_to_prove(cosmopol) # times out after at least 60 sec -- as expected

In [ ]:
#Nirvana is not derivable from other axioms
#try_to_prove(nirvana) # times out as expected after at least 60 sec

In [ ]:
try_to_find_counterexample(nirvana) # finds counterex as expected
# Same counterexample as above except that this time Bot excludes Top, Top excludes both Top and Bot.

In [ ]:
#Rashōmon does not seems to be derivable from other possibility axioms (times out at 60 sec)
#try_to_prove(rashomon)

In [ ]:
#try_to_find_counterexample(rashomon) # times out
# So if Rashomon is not a theorem, the counterexample is likely infinitely sized

In [ ]:
try_to_find_counterexample_without(rashomon,[nirvana,cosmopol,actual_world]) # finds counterexample of size 4
# So if Rashomon is a theorem, it seems to depend on nirvana, cosmopol, and actual_world

In [ ]:
#Cosmopolitanism is not derivable from other possibility axioms (counterexample found)
# So this times out as expected
#try_to_prove(cosmopol)

In [ ]:
#Cosmopolitanism is not derivable from other possibility axioms -- but there are no *finite* countermodels
# so this returns False as expected
#try_to_find_counterexample(cosmopol)

## Theorems

### Plenitude

In [ ]:
plenitude = read_expr('all x.((exists y.(-Confl(x, y) & World(y))) <-> Poss(x))')

In [ ]:
# Plenitude follows from Harmony, Cosmopol, and Symmetry. If we remove any of these, the proof search times out.
try_to_prove_without(plenitude,[nirvana, rashomon, cumulativity,complemented, dist,actual_world]) 
#not needed: nirvana, rashomon, cumulativity, complemented, dist, actual_world
# apparently needed: harmony, cosmopol, symmetry

### Distributivity of actuality

In [ ]:
#Since Actual is defined as being part of the actual world, distributivity of actuality follows just from 
# transitivity of parthood, which holds in every lattice
distofact = read_expr('all x.((exists y. (Actual(y) & Partof(x,y))) -> Actual(x))')
try_to_prove_from(distofact, lattice + [actual_def])

In [ ]:
#Find counterexample for distributivity of actuality - fails as expected
try_to_find_counterexample(distofact)

### Cumulativity of actuality

In [ ]:
#Cumulativity of actuality also follows from the basic properties of lattices
cumofact = read_expr('all x. all y.((Actual(x) & Actual(y)) -> Actual(Sum(x,y)))')
try_to_prove_from(cumofact, lattice + [actual_def])

In [ ]:
#Find counterexample for cumulativity of actuality - fails as expected
try_to_find_counterexample(cumofact)

### Manichaeism

In [ ]:
#Manichaeism depends for its proof on Harmony and Rashomon but on surprisingly little else
manichaeism = read_expr('all x.((World(x) -> all y.(Partof(y,x) | Confl(x, y))))')
try_to_prove_without(manichaeism,[actual_world,cosmopol,complemented,cumulativity,dist,nirvana,symmetry])

### Appropriateness of conjunction

In [ ]:
#Appropriateness of conjunction depends on very little: it holds in any lattice
appr_conj = read_expr(r'((exists x. (P(x) & Actual(x))) & (exists y. (Q(y) & Actual(y)))) <-> exists y exists z.(Actual(y) & Actual(z) & P(y) & Q(z) & Actual(Sum(y, z)))')
try_to_prove_from(appr_conj,lattice+[actual_def])

### Appropriateness of disjunction

In [ ]:
#Similarly, appropriateness of disjunction holds in any lattice
appr_disj = read_expr(r'(exists x. (Actual(x) & (P(x) | Q(x)))) <-> ((exists x. (Actual(x) & P(x))) | (exists y. (Actual(y) & Q(y))))')
try_to_prove_from(appr_disj,lattice+[actual_def])

### No Gaps - Lucas' version

In [ ]:
#e precludes Q iff ∃S. (e = ⊔S) ∧ (∀x∈S ∃y∈Q ∃y1⊑y, x⊥y1) ∧ (∀y∈Q ∃x∈S ∃y1⊑y, x⊥y1)

def precludes(e, Q):
    return 'exists S. ('+least_upper_bound(e,'S')+' & (all x. (S(x) -> exists y. (Q(y) & exists y1. (Partof(y1, y) & Excl(x,y1))))) & (all y. (Q(y) -> exists x (S(y) & exists y1. (Partof(y1,y) & Excl(x,y1))))))'

#example = 'exists P. ((all x1.(P(x1) -> Partof(x1,e))) & all y1.((all x1.(P(x1) -> Partof(x1,y1))) -> Partof(e,y1))) & (all x. (P(x) -> exists y. (Q(y) & exists y1. (Partof(y1, y) & Excl(x,y1))))) & (all y. (Q(y) -> exists x (P(y) & exists y1. (Partof(y1,y) & Excl(x,y1)))))'

#example_string = precludes('e','Q')
#print(example_string)
#example_expr = read_expr(example_string)
#example_expr

#define negating a set
#x negates P iff there is a function h such that x = ⊔{h(xi) ∣ xi ∈ P} and for all events 
# x1 ∈ P, h(x1) excludes some part of x1.
# 
# x negates P iff every x1 in P has a part y that excludes some part x2 of x and
# every part x3 of x
#def negate(x, P):
#    result = ('(all x1. (P(x1) -> (exists x2. Partof(x2,x) & exists y. Partof(y,x1) & Excl(x2, y)))) 
#              & (all x3. ((Partof(x3,x) & (-exists x4. (Partof(x4,x) & Partof(x4,x3)))) -> (exists x5. (P(x5) & Excl(x5, x3))))) & all x6 all x7. ((x6 != x7) -> (exists x8 exists x9.(Partof(x8,x) & Partof(x9,x) & x8!=x9 & Excl(x6, x8) & Excl(x7, x9))))')
#    return result

In [ ]:
#check no gaps
no_gaps = read_expr('(exists x. (Actual(x) & P(x))) | (exists y. (Actual(y) &'+precludes('y', 'P')+'))')
print(no_gaps)

In [ ]:
try_to_prove(no_gaps,70)

In [ ]:
try_to_find_counterexample(no_gaps)

### No Gluts - Lucas' version

In [ ]:
#check no gluts
no_gluts = read_expr('(-exists x. Actual(x) & P(x)) | (-exists y. Actual(y) &'+precludes('y', 'P')+')')
print(no_gluts)

In [ ]:
try_to_prove(no_gluts,70)

In [ ]:
try_to_find_counterexample(no_gluts)
# Finds a counterexample: Two-element model. Bot = 0, Top = 1. The actual world is Top. 
# Both Bot and Top are actual. No exclusions hold.
# Predicate P holds of both Bottom and Top. Predicate Q holds of neither.
# P holds of something actual. No Gluts would say that no actual y precludes P. 
# Since Bot and Top are actual this would mean that nothing precludes P. 
# Because of the last conjunct of the preclusion definition, indeed nothing will preclude P
# because P is nonempty and the exclusion relation is empty. 

### No Gaps - Zhuoye's version

In [ ]:
#define negating a set
#x negates P iff there is a function h such that x = ⊔{h(xi) ∣ xi ∈ P} and for all events 
# x1 ∈ P, h(x1) excludes some part of x1.
# 
# x negates P iff every x1 in P has a part y that excludes some part x2 of x and
# every part x3 of x

# def negate(x, P):
#     result = ('(all x1. (P(x1) -> (exists x2. Partof(x2,x) & exists y. Partof(y,x1) & Excl(x2, y)))) 
#               & (all x3. ((Partof(x3,x) & (-exists x4. (Partof(x4,x) & Partof(x4,x3)))) -> (exists x5. (P(x5) & Excl(x5, x3))))) & all x6 all x7. ((x6 != x7) -> (exists x8 exists x9.(Partof(x8,x) & Partof(x9,x) & x8!=x9 & Excl(x6, x8) & Excl(x7, x9))))')
#     return result


#define exclusionary negation: an event x exclusionarily negates P iff
#there is a set of events Q such that for each event x in P, part of x is excluded by some event y in Q
#and each event y in Q excludes some part of an event x in P
#and x is the sum of Q.
def exclusionary_negate(x, P):
    result = '(exists Q. ((all x1. (' + P + '(x1) -> (exists x2. Partof(x2, x1) & (exists y1. (Q(y1) & Excl(y1, x2))))))'\
             '& (all y2. (Q(y2) -> (exists x3. ('+P+'(x3) & (exists x4. (Partof(x4, x3) & Excl(y2, x4)))))))'\
              '& (all y3. (Q(y3) -> Partof(y3,'+x+'))) & (all y5. ((all y4.(Q(y4) -> Partof(y4, y5))) -> Partof('+x+', y5)))))'
    return result

#define preclude a set: an event x precludes P iff there exists a Q such that
#for any x1 in P, there is y1 in Q such that y1 excludes part of x1
#for any y2 in Q, there is x3 in P such that y2 excludes part of x3
#for any distinct y3, y4 in Q, there are distinct x5, x6 such that y3 excludes part of x5, y4 excludes part of x6
#x is the sum of Q
def preclude(x, P):
    result = '(exists Q. ((all x1.('+P+'(x1) -> (exists y1. (Q(y1) & (exists x2. (Partof(x2, x1) & Excl(y1, x2)))))))'\
             '& (all y2. (Q(y2) -> (exists x3. ('+P+'(x3) & (exists x4. (Partof(x4, x3) & Excl(y2, x4)))))))'\
             '& (all y3. (all y4. ((Q(y3) & Q(y4) & y3 != y4) -> (exists x5. (exists x6. ('+P+'(x5) & '+P+'(x6) & exists x7. (exists x8. (Partof(x7, x5) & (Partof(x8, x6) & Excl(y3, x7) & Excl(y4, x8))))))))))'\
             '& ((all y5. (Q(y5) -> Partof(y5, '+x+'))) & (all y6. ((all y7. (Q(y7) -> Partof(y7, y6))) -> Partof('+x+', y6))))))'
    return result

In [ ]:
#check no gaps with preclusion-based negation
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
prover = Prover9Command(goal, assumptions = event_frame, timeout=1000)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+')))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))

In [ ]:
#check no gaps with preclusion-based negation
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
prover = Prover9Command(goal, assumptions = event_frame, timeout=1000)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

### No Gluts - Zhuoye's version

In [ ]:
#check no gluts with exclusionary negation
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+exclusionary_negate('z', 'P')+')))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+exclusionary_negate('z', 'P')+')))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))

In [ ]:
#check no gluts with preclusion-based negation
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+preclude('z', 'P')+')))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. (-exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+preclude('z', 'P')+'))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))